## Preparation and QC of UKB participants

In [234]:
import os
import sys
import pandas as pd
import subprocess
import numpy as np
rootdir = '/home/preclineu/ramcir/Desktop/Diffusion/Subs'
#os.path.exists(rootdir)

In [235]:
def prep_sub(rootdir, id_sub):
    subdir = os.path.join(rootdir, id_sub, 'dMRI', 'dMRI')
    #Create symbolic link for data_ud with the name data
    #This is necessary for running 'quad' since all other eddy outputs are called data.eddy.-
    data_ud = os.path.join(subdir, 'data_ud.nii.gz')
    data = os.path.join(subdir, 'data.nii.gz')
    if os.path.islink(data) == False:
        os.symlink(data_ud, data)
    else:
        pass

    #Write command for creating the b0 mask and run the bash process
    fslroi_cmd = 'fslroi data.nii.gz b0_data.nii.gz 0 1'
    fslbet_cmd = 'bet b0_data.nii.gz nodif -f 0.1 -g 0 -m -n'
    prep_cmd = "cd " + subdir + " && " + fslroi_cmd + " && " + fslbet_cmd 
    subprocess.run(prep_cmd, shell = True)

    #Create aquision parameters file and index file
    #The acquisition parameters file is based on dataset acq documentation
    #i.e. AP dir = 0 1 0, PA dir = 0 -1 0, Echo spacing = 0.67 ms, EPI factor = 104 (for UKB)
    with open(os.path.join(subdir,'acqparams.txt'), 'w') as acqparams:
        acqparams.write('0 1 0 0.069\n0 -1 0 0.069')

    #The index file indicated the direction in which each volume was aquired (based on aqcparams.txt)
    #count number of volumes present in data
    with open(os.path.join(subdir,'bvals'), 'r') as bvals:
        data = bvals.read()
        vols = data.split()
        no_vols = len(vols)
    with open(os.path.join(subdir,'index.txt'), 'w') as index:
        for vol in range(0,no_vols):    
            index.write('1 ')
        

In [236]:
def quad_sub(rootdir, id_sub):  
    subdir = os.path.join(rootdir, id_sub, 'dMRI', 'dMRI')
    qc_dir = os.path.join(subdir,'data.qc') 
    #check to see if quad has already been run
    if os.path.exists(qc_dir) == False:
        #compose and run quad command
        quad_cmd = 'eddy_quad data  -idx index.txt -par acqparams.txt -m nodif_mask -b bvals -g bvecs'
        prep_cmd = "cd " + subdir + " && " + quad_cmd
        process = subprocess.run(prep_cmd, shell = True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        #save the quad folder's path to a txt file, this will be needed for squad
        with open(os.path.join(rootdir,'quad_folders.txt'), 'w') as quad_folders:
            quad_folders.write(qc_dir)
    else:
        print('QUAD already exists')     

### Run QUAD on multiple participants

In [ ]:
ready_subs = pd.read_csv(os.path.join(rootdir,'sorted_subs.csv'), usecols = ['ready'])
ready_subs = ready_subs['ready'].apply(lambda x: x if pd.isnull(x) else str(int(x))).tolist()
ready_subs = [sub for sub in ready_subs if str(sub) != 'nan']

for id_sub in ready_subs:
    print('Participant ',id_sub, 'is being processed')
    print('Preparing files')
    prep_sub(rootdir, id_sub)
    print('Running QUAD')
    quad_sub(rootdir, id_sub) 
        
print('Finished processing QUAD!')
    

In [ ]:
def quad_sub(rootdir):    
    if os.path.exists(qc_dir) == True:
        squad_cmd = 'eddy_squad'
        quad_folders = rootdir + 'quad_folders.txt'
        prep_cmd = "cd " + rootdir + " && " + quad_cmd + " && " + quad_folders
        process = subprocess.run(prep_cmd, shell = True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    else:
        print('QUAD folder not there')